In [4]:
import pandas as pd 
import numpy as np
#from surprise import Reader
#from surprise import Dataset
#from surprise.model_selection import train_test_split
#from surprise import KNNBasic
#from surprise import accuracy

In [36]:
user_inf = pd.read_csv('data/lastfm-dataset-1K/userid-profile.tsv',sep='\t')
print(user_inf.shape)
user_inf.head(10)

(992, 5)


,#id,gender,age,country,registered
0,user_000001,m,NaN,Japan,"Aug 13, 2006"
1,user_000002,f,NaN,Peru,"Feb 24, 2006"
2,user_000003,m,22.0,United States,"Oct 30, 2005"
3,user_000004,f,NaN,NaN,"Apr 26, 2006"
4,user_000005,m,NaN,Bulgaria,"Jun 29, 2006"
5,user_000006,NaN,24.0,Russian Federation,"May 18, 2006"
6,user_000007,f,NaN,United States,"Jan 22, 2006"
7,user_000008,m,23.0,Slovakia,"Sep 28, 2006"
8,user_000009,f,19.0,United States,"Jan 13, 2007"
9,user_000010,m,19.0,Poland,"May 4, 2006"


In [38]:
user_inf.isna().sum()

#id             0
gender        108
age           706
country        85
registered      8
dtype: int64

In [75]:
df2 = pd.read_csv('data/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv',
                  sep='\t',
                 #error_bad_lines = False,
                 names = ['UserID','TimeStamp','ArtID','ArtName','TraId','TraName'])


In [74]:
print(df2.shape)
df2.head()

(19098862, 6)


,UserID,TimeStamp,ArtID,ArtName,TraId,TraName
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [62]:
df2[df2['UserID'] == 'user_000001'].groupby(['UserID','TraName']).size().reset_index(name = 'Conteo').sort_values(by = 'Conteo', ascending = False)

,UserID,TraName,Conteo
1628,user_000001,Music,70
987,user_000001,Gum,63
1500,user_000001,Mario Basanov & Vidis ‘Test’,52
437,user_000001,Child Song,45
1040,user_000001,Hibari,42
...,...,...,...
1160,user_000001,If God Will Send His Angels,1
1159,user_000001,Idioteque,1
1157,user_000001,Id+Theme,1
1156,user_000001,Id+Mood,1


In [63]:
ratings = df2.groupby(['UserID','TraName']).size().reset_index(name = 'Numero_Reproducciones') 
ratings.head()

,UserID,TraName,Numero_Reproducciones
0,user_000001,'84 Pontiac Dream,14
1,user_000001,(All I Wanted Was To Make You) Feel Good,3
2,user_000001,(D)Ecaying Beauty,1
3,user_000001,(Far From) Home,1
4,user_000001,(Get Off Your) High Horse Lady,1
...,...,...,...
4407914,user_001000,“The Opening Title Sequence” (Produced By Best...,5
4407915,user_001000,“The Perfect Plan” (Produced By Best Kept Secret),4
4407916,user_001000,“The Remake Of A Remake (All I Need)” F. Tawia...,4
4407917,user_001000,“The Roots Song Wale Is On” F. Chrisette Miche...,5


In [77]:
items = df2[['TraName','TraId','ArtID','ArtName']].drop_duplicates(['TraName','TraId','ArtID','ArtName']).reset_index(drop = True)

In [82]:
items = items.assign(Id_Track = range(1, len(items)+1)) 

In [87]:
items.drop(columns = ['Id']).head()

,TraName,TraId,ArtID,ArtName,Id_Track
0,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,NaN,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,1
1,Composition 0919 (Live_2009_4_15),NaN,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,2
2,Mc2 (Live_2009_4_15),NaN,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,3
3,Hibari (Live_2009_4_15),NaN,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,4
4,Mc1 (Live_2009_4_15),NaN,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,5


In [67]:
### dataset item,user,rating
### item, descripcion
### user, name, email
1+2

3

In [ ]:
####
reader = Reader( rating_scale = ( 1, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'user_id', 'item_id', 'rating' ] ], reader )
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)